https://huggingface.co/docs/transformers/tasks/translation

In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [3]:
# !pip install --upgrade transformers

In [4]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

C:\Users\Muhammad Iqbal\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-hi'

# Helsinki-NLP/opus-mt-en-hi model

Source: https://huggingface.co/Helsinki-NLP/opus-mt-en-ur

# The Dataset

Source: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [6]:
raw_datasets = load_dataset('cfilt/iitb-english-hindi')

C:\Users\Muhammad Iqbal\anaconda3\lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [8]:
raw_datasets['train'][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

# Preprocessing the data

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

C:\Users\Muhammad Iqbal\anaconda3\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
tokenizer("Hi, this is a sentence!")

{'input_ids': [10595, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
tokenizer("Hi, this is a sentence!", "This is another sentence.")

{'input_ids': [10595, 2, 90, 23, 19, 8800, 61, 239, 23, 414, 8800, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hi, this is a sentence!", "This is another sentence."]))

{'input_ids': [[2204, 624, 2, 90, 23, 19, 44, 16, 4072, 1936, 5386, 61, 0], [44, 1687, 23, 44, 980, 9972, 581, 44, 16, 4072, 1936, 5386, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


C:\Users\Muhammad Iqbal\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    # Add labels to model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [15]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████| 2507/2507 [00:00<00:00, 2799.17 examples/s]


In [16]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [17]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [19]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [20]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [21]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle = False,
    collate_fn = data_collator,
)

In [22]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle = False,
    collate_fn = generation_data_collator,
)

In [23]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [24]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

156/156 [==============================] - 1373s 8s/step - loss: 3.7566 - val_loss: 3.9486


In [26]:
model.save_pretrained("tf_model_hi/")

# Model Testing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model_hi/")

In [ ]:
input_text = "Hi! Tell me about Pakistan"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))